In [14]:
import os, sys, json
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import matplotlib.pyplot as plt
from collections import defaultdict

In [15]:
distance_metrics = ["KL-Divergence", "Jensen-Shannon", "Wasserstein-Distance", "Energy-Distance"]

In [16]:
models = {
    "wgan": 'w_gan',
    "wgpgan": 'w_gp_gan',
    "nsgan": 'ns_gan',
    "lsgan": 'ls_gan',
    "mmgan": 'mm_gan',
    "dragan": 'dra_gan',
    "began": 'be_gan',
    "ragan": 'ra_gan',
    "infogan": 'info_gan',
    "fishergan": 'fisher_gan',
    "fgan_forward_kl": 'forkl_gan',
    "fgan_reverse_kl": 'revkl_gan',
    "fgan_jensen_shannon": 'js_gan',
    "fgan_total_var": 'tv_gan',
    "fgan_hellinger": 'hellinger_gan',
    "fgan_pearson": 'pearson_gan',
    "vae": 'vae',
    "autoencoder": 'ae',
}

In [43]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N >= batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval()
    
    def get_pred(x):
        if resize:
            x = F.interpolate(x, size=(299, 299), mode='bilinear', align_corners=False)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [48]:
%%time
print ("Calculating Inception Score...")
print (inception_score(torch.rand((128, 3, 28, 28)), cuda=False, batch_size=16, resize=True, splits=1))

Calculating Inception Score...
(1.0476466971967107, 0.0)
CPU times: user 57.7 s, sys: 18.6 s, total: 1min 16s
Wall time: 59.9 s
